In [1]:
import pandas as pd
import numpy as np
import math
import datetime

table = pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table = pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

#FIXED tiene conto delle pause pranzo
def difference2(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    
    if(time > datetime.datetime.strptime('12:00', '%H:%M') and 
       time < datetime.datetime.strptime('13:25', '%H:%M') and
       ((time2-time).seconds)/60 > 30):
        time = time + datetime.timedelta(hours=1)
        #print(time)
    return ((time2-time).seconds)/60

In [3]:

#start = dft[get_row(dft, 'Cremona, sesto 39').values]
#day='lunedi'

#indici

mean = []
#Lista dei paesi non raggiungibili
impossibile = []
for day in ['lunedi', 'venerdi']:
    row = dft[get_row(dft, table[day][0]).values]
    i1 = 0
    biases = []
    sumb = 0
    for i in range(1,len(time_table[day].dropna())):
        paese=table[day][i]
        #print(paese)
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values]
        row = dft[get_row(dft, table[day][i]).values]

        diff=difference2(day, i1, i)
        print(f"{day} {i}, diff: {diff} , value: {value.values[0][0]}")
        b = diff-value.values[0][0]
        biases.append(b)
        sumb+=diff-value.values[0][0]
        i1 = i
    mean.append(sumb/len(biases))
print(mean)
#15 minuti per carico e scarico (e caffé)

lunedi 1, diff: 10.0 , value: 5.58
lunedi 2, diff: 20.0 , value: 13.05
lunedi 3, diff: 15.0 , value: 16.97
lunedi 4, diff: 15.0 , value: 4.88
lunedi 5, diff: 20.0 , value: 17.72
lunedi 6, diff: 15.0 , value: 12.62
lunedi 7, diff: 10.0 , value: 5.97
lunedi 8, diff: 25.0 , value: 21.95
lunedi 9, diff: 20.0 , value: 11.3
lunedi 10, diff: 40.0 , value: 10.33
lunedi 11, diff: 30.0 , value: 23.88
lunedi 12, diff: 25.0 , value: 6.67
lunedi 13, diff: 25.0 , value: 16.32
venerdi 1, diff: 35.0 , value: 11.8
venerdi 2, diff: 20.0 , value: 10.82
venerdi 3, diff: 15.0 , value: 9.8
venerdi 4, diff: 20.0 , value: 9.57
venerdi 5, diff: 20.0 , value: 15.33
venerdi 6, diff: 15.0 , value: 12.77
venerdi 7, diff: 10.0 , value: 13.92
venerdi 8, diff: 30.0 , value: 10.22
venerdi 9, diff: 25.0 , value: 14.98
venerdi 10, diff: 15.0 , value: 13.78
venerdi 11, diff: 15.0 , value: 10.6
[7.904615384615386, 7.8554545454545455]


In [32]:
for day in ['lunedi', 'venerdi']:
    schedule = fur1[day]
    row = dft[get_row(dft, schedule[0]).values]
    i1 = 0
    diff_greedy = []
    diff_curr = []
    for j in range(1, len(schedule.dropna())):
        value=0
        paese = schedule[j]
        value = row.loc[:,get_column(dft, paese).values].values[0][0] + 7
        #print(f"{paese}: {value}")
        diff_greedy.append(round(value,2))
        diff=round(difference2(day, i1, j),2)
        diff_curr.append(diff)
        i1 = j
        row=dft[get_row(dft, paese).values]
    print(day)
    for j in range(len(schedule.dropna())-1):
        print(f"diff greedy: {diff_greedy[j]} , diff current: {diff_curr[j]}")
    print((round(sum(diff_greedy),2), sum(diff_curr)))

lunedi
diff greedy: 12.58 , diff current: 10.0
diff greedy: 18.77 , diff current: 20.0
diff greedy: 15.72 , diff current: 15.0
diff greedy: 18.3 , diff current: 15.0
diff greedy: 13.92 , diff current: 20.0
diff greedy: 11.88 , diff current: 15.0
diff greedy: 18.97 , diff current: 10.0
diff greedy: 13.67 , diff current: 25.0
diff greedy: 11.33 , diff current: 20.0
diff greedy: 12.97 , diff current: 40.0
diff greedy: 22.18 , diff current: 30.0
diff greedy: 27.75 , diff current: 25.0
diff greedy: 37.33 , diff current: 25.0
(235.37, 270.0)
venerdi
diff greedy: 18.2 , diff current: 35.0
diff greedy: 16.18 , diff current: 20.0
diff greedy: 12.95 , diff current: 15.0
diff greedy: 10.4 , diff current: 20.0
diff greedy: 14.68 , diff current: 20.0
diff greedy: 14.73 , diff current: 15.0
diff greedy: 13.15 , diff current: 10.0
diff greedy: 14.45 , diff current: 30.0
diff greedy: 16.92 , diff current: 25.0
diff greedy: 21.4 , diff current: 15.0
diff greedy: 11.08 , diff current: 15.0
(164.14, 220.

In [ ]:
#Get distances for every day
dfd = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
tot = 0
for day in ['lunedi', 'venerdi']:
    schedule = fur1[day]
    row = dfd[get_row(dfd, schedule[0]).values]
    tot += row.loc[:,get_column(dft, paese).values].values[0][0]
    print(f"{day}: {tot}km")